In [1]:
import sys
sys.path.append("..")

import torch
import numpy as np
import random
import torch.optim as optim

from algorithms.algorithms import *
from utils.models import *
from utils.dataloaders import *

from tabulate import tabulate

In [2]:
device = 'cpu'
dataset = SyntheticDataset(device=device)

In [3]:
def pareto_frontier_multi(myArray):
    # Sort on first dimension
    myArray = myArray[myArray[:,0].argsort()[::-1]]
    # Add first row to pareto_frontier
    pareto_frontier = myArray[0:1,:]
    # Test next row against the last row in pareto_frontier
    for row in myArray[1:,:]:
        if row[0]<pareto_frontier[-1][0] and row[1]<pareto_frontier[-1][1]:
            # If it is better on all features add the row to pareto_frontier
            pareto_frontier = np.concatenate((pareto_frontier, [row]))
        if row[0]==pareto_frontier[-1][0] and row[1]<pareto_frontier[-1][1]:
            # If it is better on all features add the row to pareto_frontier
            pareto_frontier[-1] = row
    return pareto_frontier

def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(int(seeds[i]))
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def kde_tradeoff(dataset, hp_test, seeds):
    hp = hp_test.copy()
    result = []
    result_test = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            for seed in seeds:
                c = []
                c_test = []
                hp['learning_rate'] = i
                hp['lambda_'] = k
                train, val, test = lr_kde_model_runner(dataset, hp, seeds=[seed])
                c.append(hp['learning_rate'])
                c.append(hp['lambda_'])
                c.append(train['accuracy_mean'])
                c.append(val['accuracy_mean'])
                c.append(val['ei_mean'])
                c.append(val['dp_mean'])
                c.append(val['eo_mean'])
                c.append(val['eodd_mean'])
                c_test.append(test['accuracy_mean'])
                c_test.append(test['ei_mean'])
                result_test.append(c_test)
                result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))
    return result, result_test

In [4]:
EI_hp_test = {}
EI_hp_test['lambda_'] = [0]
EI_hp_test['n_epochs'] = 100
EI_hp_test['batch_size'] = 1024 
EI_hp_test['fairness'] = ''
EI_hp_test['h'] = 0.01
EI_hp_test['delta_huber'] = 0.5
EI_hp_test['delta_effort'] = 0.5
EI_hp_test['learning_rate'] = [0.1]
EI_hp_test['lambda_'] = np.linspace(0,0.9,10)
EI_hp_test['fairness'] = 'EI'
seeds = np.arange(4)
_, results_kde = kde_tradeoff(dataset, EI_hp_test, seeds)
results_kde_pareto = pareto_frontier_multi(np.squeeze(results_kde))

training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 16.96epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:05<00:00, 17.24epochs/s]


Training finished for all seeds.
training seed 2 started


...

Training finished for all seeds.
training seed 2 started


Training: 100%|██████████| 100/100 [00:05<00:00, 17.18epochs/s]


Training finished for all seeds.
training seed 3 started


Training: 100%|██████████| 100/100 [00:05<00:00, 17.12epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ---------  ---------
            0.1        0            0.786328        0.792813  0.163427   0.172824   0.228048   0.228048
            0.1        0            0.787656        0.787813  0.13685    0.177992   0.215321   0.215321
            0.1        0            0.789766        0.77625   0.128801   0.169412   0.2035     0.2035
            0.1        0            0.785234        0.786875  0.13664    0.181862   0.222502   0.222502
            0.1        0.1          0.785703        0.790312  0.155551   0.162578   0.213788   0.213788
            0.1        0.1          0.788359        0.78875   0.125085   0.172016   0.207937   0.207937
            0.1        0.1          0.788125        0.777813  0.116634   0.160538   0.192289   0.192289
            0.1        0.1     

In [5]:
results_kde_pareto

array([[0.78175   , 0.12560454],
       [0.78125   , 0.11460479],
       [0.78075   , 0.09227994],
       [0.7795    , 0.06484556],
       [0.778     , 0.06051487],
       [0.77675   , 0.05492233],
       [0.7765    , 0.05065974],
       [0.77575   , 0.04524096],
       [0.774     , 0.03800597],
       [0.77225   , 0.02879802],
       [0.769     , 0.02022056],
       [0.76675   , 0.01818669],
       [0.76125   , 0.00450639]])

In [6]:
def lr_fb_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(int(seeds[i]))
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def fb_tradeoff(dataset, hp_test, seeds):
    hp = hp_test.copy()
    result = []
    result_test = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            for seed in seeds:
                c = []
                c_test = []
                hp['learning_rate'] = i
                hp['lambda_'] = k
                train, val, test = lr_fb_model_runner(dataset, hp, seeds=[seed])
                c.append(hp['learning_rate'])
                c.append(hp['lambda_'])
                c.append(train['accuracy_mean'])
                c.append(val['accuracy_mean'])
                c.append(val['ei_mean'])
                c.append(val['dp_mean'])
                c.append(val['eo_mean'])
                c.append(val['eodd_mean'])
                c_test.append(test['accuracy_mean'])
                c_test.append(test['ei_mean'])
                result_test.append(c_test)
                result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))
    return result, result_test

In [7]:
EI_hp_test = {}
EI_hp_test['learning_rate'] = [0.01]
EI_hp_test['n_epochs'] = 100
EI_hp_test['batch_size'] = 1024
EI_hp_test['delta_effort'] = 0.5
EI_hp_test['lambda_'] = np.linspace(0,0.25,10)
EI_hp_test['fairness'] = 'EI'

_, results_fb = fb_tradeoff(dataset, EI_hp_test, seeds)
results_fb_pareto = pareto_frontier_multi(np.squeeze(results_fb))

training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 18.74epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:05<00:00, 19.03epochs/s]


Training finished for all seeds.
training seed 2 started


...

Training finished for all seeds.
training seed 2 started


Training: 100%|██████████| 100/100 [00:05<00:00, 18.87epochs/s]


Training finished for all seeds.
training seed 3 started


Training: 100%|██████████| 100/100 [00:05<00:00, 18.44epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei          dp          eo       eodd
---------------  ---------  ----------------  --------------  ----------  ----------  ----------  ---------
           0.01  0                  0.785234        0.791875  0.164861    0.16804     0.218523    0.218523
           0.01  0                  0.786563        0.787188  0.131729    0.17339     0.220064    0.220064
           0.01  0                  0.789766        0.77875   0.139099    0.174657    0.209939    0.209939
           0.01  0                  0.785391        0.78875   0.12903     0.178525    0.220883    0.220883
           0.01  0.0277778          0.785937        0.7925    0.11533     0.137329    0.178861    0.178861
           0.01  0.0277778          0.786641        0.789375  0.0918051   0.133986    0.173237    0.173237
           0.01  0.0277778          0.789453        0.77375   0.0904395   0.137237    0.171583    0.171583
  

In [8]:
results_fb_pareto

array([[0.7795    , 0.07651949],
       [0.77775   , 0.07202773],
       [0.7725    , 0.02980499],
       [0.772     , 0.02825599],
       [0.7645    , 0.01044403],
       [0.76025   , 0.00846817],
       [0.7535    , 0.00235667],
       [0.753     , 0.0013026 ],
       [0.7505    , 0.00105907]])

In [9]:
def lr_fc_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(int(seeds[i]))
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fc_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def fc_tradeoff(dataset, hp_test, seeds):
    hp = hp_test.copy()
    result = []
    result_test = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            for seed in seeds:
                c = []
                c_test = []
                hp['learning_rate'] = i
                hp['lambda_'] = k
                train, val, test = lr_fc_model_runner(dataset, hp, seeds=[seed])
                c.append(hp['learning_rate'])
                c.append(hp['lambda_'])
                c.append(train['accuracy_mean'])
                c.append(val['accuracy_mean'])
                c.append(val['ei_mean'])
                c.append(val['dp_mean'])
                c.append(val['eo_mean'])
                c.append(val['eodd_mean'])
                c_test.append(test['accuracy_mean'])
                c_test.append(test['ei_mean'])
                result_test.append(c_test)
                result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))
    return result, result_test

In [19]:
EI_hp_test = {}
EI_hp_test['learning_rate'] = [0.01]
EI_hp_test['lambda_'] = [0]
EI_hp_test['n_epochs'] = 100
EI_hp_test['batch_size'] = 1024
EI_hp_test['delta_effort'] = 0.5
EI_hp_test['lambda_'] = 1-np.geomspace(0.001,0.999,20)
EI_hp_test['fairness'] = 'EI'
seeds = np.arange(4)
_, results_fc = fc_tradeoff(dataset, EI_hp_test, seeds)
results_fc_pareto = pareto_frontier_multi(np.squeeze(results_fc))

training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 17.67epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:05<00:00, 19.43epochs/s]


Training finished for all seeds.
training seed 2 started


...

Training finished for all seeds.
training seed 2 started


Training: 100%|██████████| 100/100 [00:05<00:00, 19.36epochs/s]


Training finished for all seeds.
training seed 3 started


Training: 100%|██████████| 100/100 [00:05<00:00, 19.26epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val           ei           dp          eo       eodd
---------------  ---------  ----------------  --------------  -----------  -----------  ----------  ---------
           0.01   0.999             0.708672        0.7125    0.0101647    0.0460024    0.080616    0.0921297
           0.01   0.999             0.715547        0.719063  0.00472195   0.0464887    0.0749176   0.103482
           0.01   0.999             0.704219        0.692813  0.0173636    0.044224     0.0742924   0.101362
           0.01   0.999             0.700781        0.696875  0.00736012   0.0601287    0.103465    0.103465
           0.01   0.998562          0.721953        0.723437  0.0139735    0.0415545    0.0774485   0.0944812
           0.01   0.998562          0.725078        0.73      0.00481891   0.040359     0.063462    0.110683
           0.01   0.998562          0.720859        0.7075    0.0115719    0.0317784    0.0

In [21]:
np.save('results/synthetic_tradeoff_fb_ei.npy',results_fb_pareto)
np.save('results/synthetic_tradeoff_fc_ei.npy',results_fc_pareto)
np.save('results/synthetic_tradeoff_kde_ei.npy',results_kde_pareto)